## Read & inspect datasets

This section adds quick checks to load a small sample from each CSV, print dtypes, and produce a chunked missing-value summary for large files. Run the cells below to get a concise report on schema, sizes, and missingness.

In [ ]:
# Cell 2: quick sample + dtype check
import pandas as pd
from pathlib import Path

dataset_dir = Path('..') / 'dataset'
files = {
    'sampled_probe': dataset_dir / 'sampled_probe.csv',
    'synthetic_probe': dataset_dir / 'synthetic_probe_data.csv',
    'traffic_counts': dataset_dir / 'synthetic_traffic_counts.csv',
    'accidents': dataset_dir / 'US_Accidents_March23.csv'
}

def sample_info(path, n=5):
    print(f"--- {path.name} ---")
    try:
        df = pd.read_csv(path, nrows=n)
        print('shape:', (len(df), len(df.columns)))
        print('dtypes:')
        print(df.dtypes)
        print('\nfirst row sample:')
        print(df.head(1).to_dict(orient='records')[0])
    except Exception as e:
        print('Error sampling:', e)

for name, p in files.items():
    sample_info(p)


In [ ]:
# Cell 3: chunked summarizer (missingness + counts)
import pandas as pd
from collections import Counter


def summarize_csv(path, usecols=None, chunksize=200_000, max_chunks=None):
    print(f'=== Summarizing {path.name} ===')
    try:
        total = 0
        nulls = Counter()
        dtypes = None
        chunks = 0
        for chunk in pd.read_csv(path, usecols=usecols, chunksize=chunksize, low_memory=False):
            chunks += 1
            total += len(chunk)
            if dtypes is None:
                dtypes = chunk.dtypes
            nulls_chunk = chunk.isna().sum()
            for col, n in nulls_chunk.items():
                nulls[col] += int(n)
            if max_chunks and chunks >= max_chunks:
                break
        print('rows inspected:', total)
        if dtypes is not None:
            print('\ndtypes (sample):')
            print(dtypes.head(20))
        print('\nTop missing columns:')
        for col, n in Counter(nulls).most_common(20):
            print(f"{col}: {n}")
    except Exception as e:
        print('Error summarizing:', e)

# Run summarizer on a few files (limited pass for accidents to avoid long runtime)
summarize_csv(files['sampled_probe'])
summarize_csv(files['traffic_counts'], max_chunks=2)
summarize_csv(files['accidents'], usecols=['ID','Start_Time','End_Time','Start_Lat','Start_Lng','End_Lat','End_Lng','Temperature(F)','Precipitation(in)','Visibility(mi)','Weather_Condition','Amenity','Traffic_Signal'], max_chunks=3)


## Cleaning: `US_Accidents_March23.csv`

This section defines a reusable cleaning function for the accidents dataset and demonstrates it on a manageable sample. The function parses datetimes, coerces numeric fields, normalizes booleans and zip codes, validates coordinates, optionally fills missing end coordinates, and documents the changes.

In [2]:
# Cleaning function for accidents dataset
import pandas as pd
import numpy as np
from typing import Iterable

BOOL_COLS = ['Amenity','Bump','Crossing','Give_Way','Junction','No_Exit','Railway','Roundabout','Station','Stop','Traffic_Calming','Traffic_Signal','Turning_Loop']
NUMERIC_COLS = ['Temperature(F)','Wind_Chill(F)','Humidity(%)','Pressure(in)','Visibility(mi)','Wind_Speed(mph)','Precipitation(in)','Distance(mi)']
DATE_COLS = ['Start_Time','End_Time','Weather_Timestamp']


def _to_bool_series(s: pd.Series) -> pd.Series:
    # Robust mapping: treats 'true','1','yes' as True; everything else False
    return s.fillna('').astype(str).str.strip().str.lower().isin({'true','1','t','yes'})


def _validate_latlng(s: pd.Series, minv: float, maxv: float) -> pd.Series:
    s = pd.to_numeric(s, errors='coerce')
    s[(s < minv) | (s > maxv)] = np.nan
    return s


def clean_accidents_df(df: pd.DataFrame, fill_end_with_start: bool = False, drop_missing_start_coords: bool = True) -> pd.DataFrame:
    """Return a cleaned copy of the accidents DataFrame.

    Steps:
    - Parse dates in DATE_COLS (coerce errors to NaT)
    - Convert NUMERIC_COLS to numeric (coerce errors to NaN)
    - Normalize boolean flag columns listed in BOOL_COLS to actual bools
    - Normalize Zipcode to first 5 digits (string)
    - Validate lat/lng ranges and optionally fill missing end coords from start coords
    - Optionally drop rows missing start coordinates (default True)

    Parameters
    ----------
    df : pd.DataFrame
        Raw dataframe loaded from CSV.
    fill_end_with_start : bool
        If True, missing End_Lat/End_Lng will be filled from Start_Lat/Start_Lng.
    drop_missing_start_coords : bool
        If True, drop rows without Start_Lat/Start_Lng.

    Returns
    -------
    pd.DataFrame
        Cleaned dataframe (copy).
    """
    df = df.copy()

    # IDs and basic types
    if 'ID' in df.columns:
        df['ID'] = df['ID'].astype(str)

    # Dates
    for c in DATE_COLS:
        if c in df.columns:
            df[c] = pd.to_datetime(df[c], errors='coerce')

    # Numbers
    for c in NUMERIC_COLS:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors='coerce')

    # Booleans
    for c in BOOL_COLS:
        if c in df.columns:
            df[c] = _to_bool_series(df[c])

    # Zipcode normalization
    if 'Zipcode' in df.columns:
        df['Zipcode'] = df['Zipcode'].astype(str).str.extract(r'(\d{5})')[0]

    # Coordinate validation
    if 'Start_Lat' in df.columns:
        df['Start_Lat'] = _validate_latlng(df['Start_Lat'], -90.0, 90.0)
    if 'Start_Lng' in df.columns:
        df['Start_Lng'] = _validate_latlng(df['Start_Lng'], -180.0, 180.0)
    if 'End_Lat' in df.columns:
        df['End_Lat'] = _validate_latlng(df['End_Lat'], -90.0, 90.0)
    if 'End_Lng' in df.columns:
        df['End_Lng'] = _validate_latlng(df['End_Lng'], -180.0, 180.0)

    # Flag for end coords
    df['has_end_coords'] = df.get('End_Lat').notna() & df.get('End_Lng').notna() if 'End_Lat' in df.columns and 'End_Lng' in df.columns else False

    # Optionally fill end with start
    if fill_end_with_start and 'Start_Lat' in df.columns and 'Start_Lng' in df.columns and 'End_Lat' in df.columns and 'End_Lng' in df.columns:
        missing_end = df['has_end_coords'] == False
        df.loc[missing_end, 'End_Lat'] = df.loc[missing_end, 'Start_Lat']
        df.loc[missing_end, 'End_Lng'] = df.loc[missing_end, 'Start_Lng']
        df['has_end_coords'] = df.get('End_Lat').notna() & df.get('End_Lng').notna()

    # Drop rows missing start coords if requested
    if drop_missing_start_coords:
        if 'Start_Lat' in df.columns and 'Start_Lng' in df.columns:
            before = len(df)
            df = df.dropna(subset=['Start_Lat','Start_Lng'])
            # small log
            print(f'Dropped {before - len(df)} rows with missing start coordinates')

    # Add simple derived columns if applicable
    if 'Start_Time' in df.columns:
        df['start_hour'] = df['Start_Time'].dt.hour
        df['start_date'] = df['Start_Time'].dt.date

    return df


In [3]:
# Demo: run cleaning on a sample and save cleaned sample
import pandas as pd
from pathlib import Path

acc_path = Path('..') / 'dataset' / 'US_Accidents_March23.csv'
out_path = Path('..') / 'dataset' / 'accidents_sample_cleaned'

# Choose manageable sample size for quick runs
nrows = 100_000
print('Loading sample rows:', nrows)
raw = pd.read_csv(acc_path, nrows=nrows)
print('Raw missing top columns:')
print(raw.isna().sum().sort_values(ascending=False).head(10))

cleaned = clean_accidents_df(raw, fill_end_with_start=False)
print('\nAfter cleaning, top missing columns:')
print(cleaned.isna().sum().sort_values(ascending=False).head(10))

print('\nBoolean flags sample (counts):')
for c in BOOL_COLS:
    if c in cleaned.columns:
        print(f"{c}:", cleaned[c].value_counts(dropna=False).to_dict())

# Save cleaned sample (try parquet first, fall back to csv)
out_parquet = out_path.with_suffix('.parquet')
out_csv = out_path.with_suffix('.csv')

try:
    cleaned.to_parquet(out_parquet)
    print('Saved cleaned sample to', out_parquet)
except Exception as e:
    print('Parquet write failed (missing dependency?), falling back to CSV. Error:', e)
    cleaned.to_csv(out_csv, index=False)
    print('Saved cleaned sample to', out_csv)


Loading sample rows: 100000
Raw missing top columns:
End_Lng              100000
End_Lat              100000
Wind_Chill(F)         95678
Precipitation(in)     92632
Wind_Speed(mph)       23820
Humidity(%)            1856
Visibility(mi)         1846
Weather_Condition      1604
Temperature(F)         1591
Pressure(in)           1292
dtype: int64
Dropped 0 rows with missing start coordinates

After cleaning, top missing columns:
End_Lat              100000
End_Lng              100000
Wind_Chill(F)         95678
Precipitation(in)     92632
Wind_Speed(mph)       23820
Humidity(%)            1856
Visibility(mi)         1846
Weather_Condition      1604
Temperature(F)         1591
Pressure(in)           1292
dtype: int64

Boolean flags sample (counts):
Amenity: {False: 99216, True: 784}
Bump: {False: 99952, True: 48}
Crossing: {False: 93511, True: 6489}
Give_Way: {False: 99834, True: 166}
Junction: {False: 89566, True: 10434}
No_Exit: {False: 99946, True: 54}
Railway: {False: 98738, True: 1262

In [3]:
# Run full end-to-end cleaning pipeline (robust execution)
print('Running full pipeline: this may take several minutes depending on file size and system resources')
import runpy
import os
from pathlib import Path

# ensure we run from project root so relative paths inside the script work
cwd = Path.cwd()
project_root = cwd.parent  # notebooks/.. => project root
os.chdir(project_root)
try:
    print('Starting pipeline... (this will print progress)')
    runpy.run_path('tools/accidents_pipeline.py', run_name='__main__')
    print('Pipeline finished')
finally:
    os.chdir(cwd)
    print('Restored working directory')


Running full pipeline: this may take several minutes depending on file size and system resources
Starting pipeline... (this will print progress)
Computing city-level medians for weather columns (this may take a while)...
Computed medians from 7728394 rows: overall medians sample: {'Temperature(F)': np.float64(68.0), 'Humidity(%)': np.float64(63.0), 'Pressure(in)': np.float64(29.95)}
Processed 500000 rows...
Processed 1000000 rows...
Processed 1500000 rows...
Processed 2000000 rows...
Processed 2500000 rows...
Processed 3000000 rows...
Processed 3500000 rows...
Processed 4000000 rows...
Processed 4500000 rows...
Processed 5000000 rows...
Processed 5500000 rows...
Processed 6000000 rows...
Processed 6500000 rows...
Processed 7000000 rows...
Processed 7500000 rows...
Processed 7728394 rows...
Pipeline complete. Output written to dataset\cleand-data\US_Accidents_March23_cleaned.csv
Pipeline finished
Restored working directory
